# lab assesment


In [1]:
import numpy as np
import pandas as pd
from numpy import log2 as log
from sklearn import tree
import pydotplus
from IPython.display import Image
import time

In [2]:
eps = np.finfo(float).eps

In [12]:
dataset = {'Green':['n','y','y','n','y','n','n','n','y','n' ],
            'legs':[3,2,3,2,3,2,2,2,2,2],
            'Height':['s','t','t','s','t','t','s','t','s','t'],
            'Smelly':['y','n','n','y','y','y','n','n','n','y'],
           'Species':['m','m','m','m','m','h','h','h','h','h' ]
          }

In [14]:
df = pd.DataFrame(dataset, columns=['Green','legs','Height','Smelly','Species'])
print(df)

  Green  legs Height Smelly Species
0     n     3      s      y       m
1     y     2      t      n       m
2     y     3      t      n       m
3     n     2      s      y       m
4     y     3      t      y       m
5     n     2      t      y       h
6     n     2      s      n       h
7     n     2      t      n       h
8     y     2      s      n       h
9     n     2      t      y       h


In [15]:
#q1
entropy_node = 0
values = df.Species.unique()
for value in values:
    fraction = df.Species.value_counts()[value]/len(df.Species)
    entropy_node += -fraction * np.log2(fraction)
    
print('Parent Entropy is: ', entropy_node)

Parent Entropy is:  1.0


In [21]:
def index(attribute):
    attribute = attribute
    target_variables  = df.Species.unique()
    variables = df[attribute].unique()
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df.Species == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy_each_feature += fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy_attribute += -fraction2*entropy_each_feature

    IG_attribute = entropy_node - abs(entropy_attribute)
    print("Entropy of ", attribute, "is: ", round(abs(entropy_attribute),3))
    print("IG ", "due to", attribute, "is: ", round(IG_attribute,3))
    
# index('Smelly')
# index('Green')
# index('legs')
# index('Height')

In [25]:
#q2
x_columns = pd.get_dummies(df[['Green','legs','Height','Smelly','Species']])

for rand in range(1):
    dtree = tree.DecisionTreeClassifier(max_features = 2, random_state = rand, max_depth = 6, 
                                        criterion = 'gini', splitter = 'best')
    model = dtree.fit(x_columns,df['Species'])
    out_file = tree.export_graphviz(model,feature_names = list(x_columns.columns.values), class_names = ['M','H'], filled = True, rounded = True)
    graph = pydotplus.graph_from_dot_data(out_file)
    Image(graph.create_png())
    graph.write_png('tree2.png')
    print(rand)
#     time.sleep(5)
    
    

0


In [8]:
#q3
# fraction=0
import pprint
def find_entropy(df):
    global fraction
    Class = df.keys()[-1]
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction += df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
        

In [9]:
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]
    target_variables = df[Class].unique()
    variables = df[attribute].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)


In [10]:
def find_winner(df):
    IG=[]
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]


In [11]:
def get_subtable(df, node, value):
    return df[df[node]==value].reset_index(drop=False)

In [12]:
def buildTree(df,d_tree=None):
    node = find_winner(df)
    attValue = np.unique(df[node])
#     attValue ['asd','xycdf']
    
    if d_tree is None:
        d_tree={}
        d_tree[node] = {}
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Species'],return_counts=True)
#         print(len(counts))
#         print(subtable)
        if len(counts)==1:
            d_tree[node][value] = clValue[0]
            
        else:
#             print(value)
            d_tree[node][value] = buildTree(subtable)
    return d_tree

dtree = buildTree(df)
print("\n\n The decision tree in dictionary form is ")
pprint.pprint(dtree)



 The decision tree in dictionary form is 
{'Taste': {'Salty': {'index': {0: 'No', 6: 'No', 9: 'Yes'}},
           'Spicy': {'index': {1: 'No', 2: 'Yes', 3: 'No', 4: 'Yes', 8: 'Yes'}},
           'Sweet': 'Yes'}}
